In [1]:
#import libraries
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification ,AutoTokenizer, AutoConfig
import numpy as np
#convert logits to probabilities
from scipy.special import softmax

Set model paths

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

model_path = f"Junr-syl/tweet_sentiments_analysis"
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

c:\Users\sylvester\Desktop\Azubi data analyst\career accelerator_clones\LP2\NLP-Sentiment Analysis\venv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sylvester\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Create a function to hide user ids in text

In [4]:
def preprocess(text):
    #initiate an empty list 
    new_text = []
    #split text by space
    for t in text.split(" "):
        #set username to @user
        t = '@user' if t.startswith('@') and len(t) > 1 else t  
        #set tweet source to http
        t = 'http' if t.startswith('http') else t 
        #store text in the list
        new_text.append(t)
        #change text from list back to string
    return " ".join(new_text) 

Make predictions

In [5]:
# Input preprocessing
text = "Covid cases are increasing fast!"
text = preprocess(text)

# PyTorch-based models
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

In [6]:
#change label id 
config.id2label = {0: 'NEGATIVE', 1: 'NEUTRAL', 2: 'POSITIVE'}


In [7]:
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) NEUTRAL 0.9856
2) POSITIVE 0.0097
3) NEGATIVE 0.0047
